# CSC4008 Tut 10
## Collaborative Filtering

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [ ]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [ ]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [ ]:
# YOUR CODE HERE
##
print("Number of rows:", training.count())
print("Columns:",training.columns)
training.groupby("rating").count().show()
training.select("rating").count()

Number of rows: 80000
Columns: ['user_id', 'item_id', 'rating', 'timestamp']
+------+-----+
|rating|count|
+------+-----+
|     1| 4719|
|     3|21963|
|     5|16744|
|     4|27396|
|     2| 9178|
+------+-----+



80000

Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [ ]:
# YOUR CODE HERE
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

Now compute the RMSE on the test dataset.


In [ ]:
# YOUR CODE HERE
predictions = model.transform(test)
predictions.show()
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

+-------+-------+------+---------+----------+
|user_id|item_id|rating|timestamp|prediction|
+-------+-------+------+---------+----------+
|    148|      1|     4|877019411| 3.9755447|
|    148|      7|     5|877017054| 3.0967607|
|    148|     70|     5|877021271|  2.139257|
|    148|     71|     5|877019251| 3.7885785|
|    148|     78|     1|877399018| 3.9618964|
|    148|     98|     3|877017714|   4.01637|
|    148|    114|     5|877016735| 6.2282047|
|    148|    116|     5|877398648| 1.3983161|
|    148|    140|     1|877019882| -0.363299|
|    148|    163|     4|877021402|0.28191411|
|    148|    169|     5|877020297| 4.6284156|
|    148|    172|     5|877016513| 4.1573296|
|    148|    177|     2|877020715|  4.133243|
|    148|    185|     1|877398385| 1.9688009|
|    148|    204|     3|877016912|  5.111382|
|    148|    214|     5|877019882|  4.088899|
|    148|    228|     4|877016514| 4.7109323|
|    148|    357|     5|877016735|  4.584553|
|    148|    408|     5|877399018|

At this point, you can use the trained model to produce the top-K recommendations for each user.

In [ ]:
# YOUR CODE HERE
K = 10
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)

userRecs.select("user_id","recommendations").show(10)
print(userRecs.select("user_id","recommendations").collect())
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)



+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      1|[{960, 6.5479207}...|
|      3|[{390, 8.685715},...|
|      5|[{853, 7.591122},...|
|      6|[{1643, 6.0243444...|
|      9|[{1085, 13.354608...|
|     12|[{1262, 8.844548}...|
|     13|[{842, 7.099153},...|
|     15|[{1426, 9.6538515...|
|     16|[{1131, 7.8392797...|
|     17|[{1066, 8.634191}...|
+-------+--------------------+
only showing top 10 rows

[Row(user_id=1, recommendations=[Row(item_id=960, rating=6.5479207038879395), Row(item_id=1005, rating=6.510010719299316), Row(item_id=1113, rating=6.50874662399292), Row(item_id=1643, rating=6.463581085205078), Row(item_id=1449, rating=6.331055641174316), Row(item_id=1131, rating=6.3034515380859375), Row(item_id=454, rating=6.1868133544921875), Row(item_id=916, rating=6.159509658813477), Row(item_id=1251, rating=6.025094509124756), Row(item_id=921, rating=6.005616664886475)]), Row(user_id=3, recommendations=[Row(item_id=390, rating=8